In [118]:
import glob, os, re, numpy as np, pandas as pd
import shutil
from string import Template

In [119]:
items = pd.read_csv('wsb-catalog.csv').fillna('')

In [120]:
items.head(20)

,identifier,creator,title,publisher,date,bibliographicCitation,description
0,junkie-1,William S. Burroughs,Junkie,"New York: Ace Books, Inc.",1953,"schottlaender/4.0/A1a, maynard-and-miles/A1a",First. Not as much yellowing as is typical.
1,junkie-2,William S. Burroughs,Junkie,"New York: Ace Books, Inc.",1953,"schottlaender/4.0/A1a, maynard-and-miles/A1a","First. Signed by Burroughs, dated ""Nov. 6, 198..."
2,junkie-3,William S. Burroughs,Junkie,London: Digit Books,1957,"schottlaender/4.0/A1b, maynard-and-miles/A1b",First English. In custom clamshell case.
3,british-journal-of-addiction-5-1,William S. Burroughs,British Journal of Addiction 53(2),,1957-01,"schottlaender/4.0/C3, maynard-and-miles/C1b","Offprint of Burroughs' article ""Letter From A ..."
4,chicago-review-12-3-1,William S. Burroughs,Chicago Review 12(3),,1958,"schottlaender/4.0/C5, maynard-and-miles/C5",Soiled and worn on the spine.
5,the-naked-lunch-1,William S. Burroughs,The Naked Lunch,Paris: The Olympia Press,1959,"schottlaender/4.0/A2a, maynard-and-miles/A2a",First. Signed by Burroughs. With slight chippi...
6,big-table-1-1,William S. Burroughs,Big Table 1,,1959,"schottlaender/4.0/C7, maynard-and-miles/C6",
7,big-table-2-1,William S. Burroughs,Big Table 2,,1959,"schottlaender/4.0/C8, maynard-and-miles/C7",
8,new-departures-1-1,William S. Burroughs,New Departures 1,,1959,"schottlaender/4.0/C11, maynard-and-miles/C8","Signed by Burroughs at second entry ""2: Coke B..."
9,the-exterminator-1,William S. Burroughs,The Exterminator,San Francisco: The Auerhahn Press,1960,"schottlaender/4.0/A3a, maynard-and-miles/A4a",First. Yellowing on edges.


In [130]:
template = Template("""
title = "$title"
authors = $authors
date = 1970-01-01
publication_types = ['5']
abstract = "$abstract"
tags = $tags
featured = false

[ld]

"@id" = "https://wsb-bibliography.github.io/wsb-catalog-v2/$identifier"
"@type" = ["http://id.loc.gov/ontologies/bibframe/Item", "http://id.loc.gov/ontologies/bibframe/Text"]
itemOf = "https://wsburroughs.link/anything-but-routine/4.0/instance/instance"

[ld."@context"]
    [ld."@context".itemOf]
    "@id" = "http://id.loc.gov/ontologies/bibframe/itemOf"
    "@type" = "@id"
    [ld."@context".label]
    "@id" = "http://www.w3.org/2000/01/rdf-schema#label"
    [ld."@context".role]
    "@id" = "http://id.loc.gov/ontologies/bibframe/role"
    [ld."@context".value]
    "@id" = "http://www.w3.org/1999/02/22-rdf-syntax-ns#value"
    [ld."@context".heldBy]
    "@id" = "http://id.loc.gov/ontologies/bibframe/heldBy"
    "@type" = "@id"
    [ld."@context".note]
    "@id" = "http://id.loc.gov/ontologies/bibframe/note"
    "@type" = "@id"

[ld.heldBy]
"@type" = ["http://id.loc.gov/ontologies/bibframe/Agent", "http://id.loc.gov/ontologies/bibframe/Person"]
label = "Bradley P. Allen"
role = "collector"

[ld.note]
"@type" = ["http://id.loc.gov/ontologies/bibframe/Note", "http://id.loc.gov/ontologies/bibframe/DescriptiveNote"]
value = "$abstract"
""")

In [131]:
def generate_front_matter(record, template, outfile=None):
    record['title'] = record['title'].replace('"', r'\"')
    record['authors'] = [author for author in record['creator'].split(', ')]
    if record['description'] != "":
        description = record['description'].replace('"', r'\"')
    else:
        description = ""
    if record['publisher'] != "":
        publisher = record['publisher'].replace('"', r'\"')  
    else:
        publisher = "n.p."
    record['abstract'] = f"{publisher}, {record['date']}. {description}".strip()
    if record['bibliographicCitation'] != "":
        record['tags'] = [tag for tag in record['bibliographicCitation'].split(', ')] 
    else:
        record['tags'] = []
    print("+++", file=outfile)
    print(template.substitute(record), file=outfile)
    print("+++", file=outfile)

In [132]:
for record in records:
    dir = f"content/publication/{record['identifier']}"
    md_file = dir + "/index.md"
    source_img_file = f"assets/images/wsb-items/{record['identifier']}.jpg"
    target_img_file = dir + "/featured.jpg"
    os.mkdir(dir)
    shutil.copy(source_img_file, target_img_file)
    with open(md_file, 'w') as outfile:
        generate_front_matter(record, template, outfile)